# Predicting NYC apartment's value with open data

**tldr;** Boldly predict a NYC apartments market value using only public / open data. Check out the [Price Predictor](#Price-Predictor).


Any good brokers in NYC will use a variety of factors to determine the value of your apartment, including:

1. Recent sales in your building / Neighbourhood
2. Square footage
3. Renovation status
4. View, closeness to subway, # of bedrooms etc. 

Unfortunately this data, is not easily available for the purposes of analysis. Note: it's available on Streeteasy and other REBNY memebers but not easily available for data analysis.

Here I present an alternative approach to pricing NYC apartments with only public data from [NYC Open Data](https://opendata.cityofnewyork.us/). 


In [1]:
import numpy as np
import pandas as pd
import datetime
from scipy.spatial import distance

import ipywidgets as widgets
from ipywidgets import IntSlider

from datascience import *
from datetime import timedelta
from datetime import date
from datetime import datetime
import time

from IPython.display import display
from IPython.display import HTML

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')


import locale
%load_ext line_profiler
import os
import xlrd as xlrd

# Constants

Let's define some constants to help make our code more readable

In [2]:


# Constants
raw_directory = "input/"
output_directory = "output/"


# Declare column names to allow for auto completion :)

COL_SALE_DATE = 'SALE DATE'

COL_PURCHASE_DATE = 'PURCHASE DATE'
COL_SOLD_DATE = 'SOLD DATE'
COL_PURCHASE_PRICE = 'PURCHASE PRICE'
COL_SOLD_PRICE = 'SOLD PRICE'
COL_FULL_ADDRESS = 'FULL ADDRESS'
COL_PRICE_CHANGE = 'PRICE CHANGE'
COL_PERIOD = 'PERIOD'

COL_SALE_YEAR = 'SALE_YEAR'
COL_SALE_MONTH = 'SALE_MONTH'
COL_SALE_PRICE = 'SALE PRICE'


# Daily price change column name
COL_DAILY_PRICE_CHANGE = 'DAILY PRICE CHANGE'

COL_PURCHASE_DATE_SU = 'PURCHASE DATE SU'
COL_PURCHASE_PRICE_SU = 'PURCHASE PRICE SU'
COL_SOLD_DATE_SU = 'SOLD DATE SU'




# Functions

Also lets define some functions to assist in our analysis

In [3]:
# def original_value(original_values, sd_value ):
#     '''convert a standard unit value to it's original value'''

def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)


def correlation(t, x, y):
    return np.mean(standard_units(t.column(x)) * standard_units(t.column(y)))


def slope(table, x, y):
    r = correlation(table, x, y)
    return r * np.std(table.column(y)) / np.std(table.column(x))


def intercept(table, x, y):
    a = slope(table, x, y)
    return np.mean(table.column(y)) - a * np.mean(table.column(x))


def fit(table, x, y):
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * table.column(x) + b


def residual(table, x, y):
    return table.column(y) - fit(table, x, y)


def scatter_fit(table, x, y):
    plots.scatter(table.column(x), table.column(y), s=20)
    plots.plot(table.column(x), fit(table, x, y), lw=2, color='gold')
    plots.xlabel(x)
    plots.ylabel(y)


# Helpers Functions

def print_stats(data):
    '''Prints common stats for a data array'''

    data_mean = np.mean(data)
    data_std = np.std(data)
    data_min = min(data)
    data_max = max(data)

    percent_5 = percentile(5, data)
    percent_95 = percentile(95, data)
    percent_1 = percentile(1, data)
    percent_99 = percentile(99, data)

    percent_25 = percentile(25, data)
    percent_50 = percentile(50, data)
    percent_75 = percentile(75, data)

    print("Avg:", data_mean, "\tStd:", data_std, "\tMin:", data_min, "\tMax:", data_max)
    print(" 5%:", percent_5, "\t95%:", percent_95)
    print(" 1%:", percent_1, "\t99%:", percent_99)
    print("25%:", percent_25, "\t50%:", percent_50, '\t75%', percent_75)


def print_col_stats(table, col_name):
    ''' Print the stats For column named'''

    print(col_name, "Stats")
    data = table.column(col_name)
    print_stats(data)


def draw_hist(table: Table, col_name, offset_percent=0):
    ''' Draw a histogram for table with an additional offset percent'''
    data = table.column(col_name)
    offset_start = percentile(offset_percent, data)
    offset_end = percentile(100 - offset_percent, data)
    table.hist(col_name, bins=np.arange(offset_start, offset_end, (offset_end - offset_start) / 20))


def col_stats(table, col_name):
    ''' Prints state for a column in table'''
    print_col_stats(table, col_name)
    draw_hist(table, col_name)


# Step 1: Import data

Sources
1. [Annualized Sales Data](https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page)
2. [Rolling Sale data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [4]:
# List data files and directories in current directory
excel_files = os.listdir(raw_directory)

# Select only tje xls files
excel_files = [k for k in excel_files if '.xls' in k]

In [5]:
excel_files[:len(excel_files)]

['2015_manhattan.xls',
 '2011_manhattan.xls',
 '2016_manhattan.xls',
 '2012_manhattan.xls',
 'rollingsales_manhattan.xls',
 'sales_manhattan_03.xls',
 'sales_manhattan_06.xls',
 'sales_manhattan_04.xls',
 'sales_2007_manhattan.xls',
 'sales_manhattan_05.xls',
 '2009_manhattan.xls',
 '2013_manhattan.xls',
 '2017_manhattan.xls',
 'sales_2008_manhattan.xls',
 '2010_manhattan.xls',
 '2014_manhattan.xls']

Unfortunately not all files are formatted the same way. Some have the header in row 4, others in row 5. We can check by making sure 'BOROUGH' is the first column in the imported dataset 

In [6]:

# Create an data store
all_sales_data = pd.DataFrame()

# Load individual excel files. 
for excel_file in excel_files:
    print(excel_file)
    
    # Read excel, Note the headers could in row 4 or row 5 (index=3 or 4). 
    yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=3, encoding='sys.getfilesystemencoding()')
   
    # Check if the first column is "BOROUGH"
    if not yearly_sales_data.columns[0].startswith('BOROUGH'):
        # Otherwise the data starts from row 5.
         yearly_sales_data = pd.read_excel(raw_directory+excel_file, header=4, encoding='sys.getfilesystemencoding()')
    
#     print(yearly_sales_data.shape)
    
    yearly_sales_data.rename(columns=lambda x: x.strip(), inplace=True)
    
    all_sales_data = all_sales_data.append(yearly_sales_data)
    
#     print(all_sales_data.shape)

2015_manhattan.xls
2011_manhattan.xls
2016_manhattan.xls
2012_manhattan.xls
rollingsales_manhattan.xls
sales_manhattan_03.xls
sales_manhattan_06.xls
sales_manhattan_04.xls
sales_2007_manhattan.xls
sales_manhattan_05.xls
2009_manhattan.xls
2013_manhattan.xls
2017_manhattan.xls


/Users/somya/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


sales_2008_manhattan.xls
2010_manhattan.xls
2014_manhattan.xls


Spot check to verify data

In [7]:

all_sales_data.sample(10)


,ADDRESS,APARTMENT NUMBER,BLOCK,BOROUGH,BUILDING CLASS AS OF FINAL ROLL 17/18,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,EASE-MENT,...,NEIGHBORHOOD,RESIDENTIAL UNITS,SALE DATE,SALE PRICE,TAX CLASS AS OF FINAL ROLL 17/18,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE,TOTAL UNITS,YEAR BUILT,ZIP CODE
12236,"210 EAST 73RD STREET, 5E",,1427,1,D4,NaN,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (59-79),0,2017-02-23,845000,2,NaN,2,0,1929,10021
8822,335-41 EAST 51 STREET,7H,1344,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,MIDTOWN EAST,1,2003-12-16,272950,NaN,2,2,1,0,10022
8410,"310 EAST 46TH STREET, 14K",,1338,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,MIDTOWN EAST,0,2016-07-26,745000,NaN,2,2,0,1929,10017
12338,"35 EAST 85TH STREET, 2C",,1497,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (79-96),0,2018-03-14,2850000,NaN,2,2,0,1959,10028
6194,50 CLINTON STREET,STORB,344,1,NaN,RS,RS,47 CONDO NON-BUSINESS STORAGE,0,,...,LOWER EAST SIDE,0,2018-03-20,0,NaN,4,4,1,2015,10002
3259,280 PARK AVENUE SOUTH,11C,850,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,FLATIRON,1,2013-06-14,1250000,NaN,2,2,1,1986,10010
13410,108 WEST 57TH STREET,,1009,1,NaN,,H2,25 LUXURY HOTELS,0,,...,MIDTOWN WEST,0,2008-02-09,0,NaN,,4,0,0,10019
18254,923 5 AVENUE,12E,1388,1,NaN,R4,R4,13 CONDOS - ELEVATOR APARTMENTS,0,,...,UPPER EAST SIDE (59-79),1,2005-03-15,0,NaN,2,2,1,1982,10021
9362,"411 EAST 57TH STREET, 18D",,1369,1,NaN,D4,D4,10 COOPS - ELEVATOR APARTMENTS,0,,...,MIDTOWN EAST,0,2014-05-01,780000,NaN,2,2,0,1959,10022
5128,61 LEXINGTON AVENUE,G,881,1,NaN,R5,R5,28 COMMERCIAL CONDOS,1,,...,GRAMERCY,0,2007-01-30,350000,NaN,4,4,1,0,10010


In [8]:
# Check for duplicate entries
sum(all_sales_data.duplicated(all_sales_data.columns))

16156

In [9]:
#Delete the duplicates and check that it worked
all_sales_data = all_sales_data.drop_duplicates(all_sales_data.columns, keep='last')
sum(all_sales_data.duplicated(all_sales_data.columns))

0

# Step 2. Clean up the data and arrange for analysis

In [10]:
#SALE DATE is object but should be datetime
all_sales_data[COL_SALE_DATE] = pd.to_datetime(all_sales_data[COL_SALE_DATE], errors='coerce')
all_sales_data['APARTMENT NUMBER'] = all_sales_data['APARTMENT NUMBER'].astype(str)

# remove additional whitespace in strings
all_sales_data = all_sales_data.applymap(lambda x: x.strip() if type(x) is str else x)

In [11]:
# Convert to Table for analysis
all_sales_data = Table.from_df(all_sales_data)

In [12]:
# remove and sales less than $10,000 they are likely to be a property transfer rather than an actual sale
all_sales_data = all_sales_data.where(COL_SALE_PRICE, are.above(10000))

In [13]:
# SPOT CHECK: Understand the data labels we might be interested in
all_sales_data.labels

('ADDRESS',
 'APARTMENT NUMBER',
 'BLOCK',
 'BOROUGH',
 'BUILDING CLASS AS OF FINAL ROLL 17/18',
 'BUILDING CLASS AT PRESENT',
 'BUILDING CLASS AT TIME OF SALE',
 'BUILDING CLASS CATEGORY',
 'COMMERCIAL UNITS',
 'EASE-MENT',
 'GROSS SQUARE FEET',
 'LAND SQUARE FEET',
 'LOT',
 'NEIGHBORHOOD',
 'RESIDENTIAL UNITS',
 'SALE DATE',
 'SALE PRICE',
 'TAX CLASS AS OF FINAL ROLL 17/18',
 'TAX CLASS AT PRESENT',
 'TAX CLASS AT TIME OF SALE',
 'TOTAL UNITS',
 'YEAR BUILT',
 'ZIP CODE')

In [14]:
# Remove columns we don't actually need. e.g. lot, block etc

all_sales_data = all_sales_data.select(['SALE DATE', 'SALE PRICE', 'ADDRESS','APARTMENT NUMBER', 'YEAR BUILT', 'NEIGHBORHOOD', 'ZIP CODE', 'BUILDING CLASS AT TIME OF SALE', 'BUILDING CLASS CATEGORY'])

In [15]:
# Combine the Address and Apt No. The data here seems a little messy. Sometime the address itself contains the apt no,
# other times it's seperated. Let's create a full address column that combines them into a single address

def combine_address(address, aptNo):
    """Combine the address and Apartment into a single result"""
    temp = address.strip()
    if len(aptNo.strip()) > 0:
        temp = temp + ', ' + aptNo.strip()
    return temp

full_address = all_sales_data.apply(combine_address, ['ADDRESS', 'APARTMENT NUMBER'])

# Add a Full Address column
all_sales_data =  all_sales_data.with_column(COL_FULL_ADDRESS, full_address)

/Users/somya/Library/Python/3.7/lib/python/site-packages/datascience/tables.py:361: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)


In [16]:
# Try to understand Building Class Codes. What are the most common Codes?

all_sales_data.group(['BUILDING CLASS AT TIME OF SALE', 'BUILDING CLASS CATEGORY']).sort('count', descending=True).show(20)

# Ok mostly condos and co-ops as expected

# Question what's R5 -  COMMERCIAL CONDOS? ignoring for now

BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,count
R4,13 CONDOS - ELEVATOR APARTMENTS,90128
D4,10 COOPS - ELEVATOR APARTMENTS,83556
R5,28 COMMERCIAL CONDOS,15569
R4,13 CONDOS - ELEVATOR APARTMENTS,10777
C6,09 COOPS - WALKUP APARTMENTS,10413
D4,10 COOPS - ELEVATOR APARTMENTS,10319
R9,17 CONDOPS,10097
H2,25 LUXURY HOTELS,6204
R9,17 CONDO COOPS,4148
RH,45 CONDO HOTELS,4011


Let's pick out the condos for now. 
Reference Data: [NYC Building Codes](https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html) 

In [17]:
condos = all_sales_data.where('BUILDING CLASS AT TIME OF SALE', are.contained_in("R1R2R3R4R6"))

# Spot Check condo data
condos.sample(10)

SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,FULL ADDRESS
2008-10-29 00:00:00,1731371,200 EAST 66TH STREET,C0502,1951,UPPER EAST SIDE (59-79),10065,R4,13 CONDOS - ELEVATOR APARTMENTS,"200 EAST 66TH STREET, C0502"
2010-04-05 00:00:00,3450000,252 SEVENTH AVENUE,11P,0,CHELSEA,10001,R4,13 CONDOS - ELEVATOR APARTMENTS,"252 SEVENTH AVENUE, 11P"
2013-08-14 00:00:00,560000,250 EAST 54TH STREET,20B,1990,MIDTOWN EAST,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"250 EAST 54TH STREET, 20B"
2011-12-19 00:00:00,736000,99 JOHN STREET,2311,2003,FINANCIAL,10038,R4,13 CONDOS - ELEVATOR APARTMENTS,"99 JOHN STREET, 2311"
2004-04-30 00:00:00,645000,150 NASSAU STREET,3H,0,SOUTHBRIDGE,10038,R4,13 CONDOS - ELEVATOR APARTMENTS,"150 NASSAU STREET, 3H"
2003-10-08 00:00:00,235000,520 EAST 81 STREET,2M,1964,UPPER EAST SIDE (79-96),10028,R4,13 CONDOS - ELEVATOR APARTMENTS,"520 EAST 81 STREET, 2M"
2017-12-12 00:00:00,1150000,300 RECTOR PLACE,4Q,0,TRIBECA,10280,R4,13 CONDOS - ELEVATOR APARTMENTS,"300 RECTOR PLACE, 4Q"
2016-10-28 00:00:00,1035000,372 CENTRAL PARK WEST,17V,1961,MANHATTAN VALLEY,10025,R4,13 CONDOS - ELEVATOR APARTMENTS,"372 CENTRAL PARK WEST, 17V"
2017-03-23 00:00:00,1730000,101 WEST 24TH STREET,29H,2006,CHELSEA,10011,R4,13 CONDOS - ELEVATOR APARTMENTS,"101 WEST 24TH STREET, 29H"
2015-12-18 00:00:00,5500000,500 PARK AVENUE,19B,0,MIDTOWN CBD,10022,R4,13 CONDOS - ELEVATOR APARTMENTS,"500 PARK AVENUE, 19B"


Let's find condos with multiple sales

In [18]:
condos.group(COL_FULL_ADDRESS).sort(1, descending=True)

FULL ADDRESS,count
1 WEST END AVENUE,136
45 EAST 22ND STREET,51
245 WEST 99TH STREET,51
171 WEST 131 STREET,44
959 1 AVENUE,42
416 WEST 52ND STREET,39
245 WEST 99TH STREET,37
2094 FREDRICK DOUGLASS BL,35
172 MADISON AVENUE,35
360 EAST 89TH STREET,23


It looks like apt numbers are missing for most of the top sales. In order to focus on manhattan apartments let's ignore anything without an apt number. i.e. anything without a ',' (comma) in the Full Address

In [ ]:
multi_sale_condos = condos.where(COL_FULL_ADDRESS, are.containing(',')).group(COL_FULL_ADDRESS).sort(1, descending=True).where('count', are.above(1))
multi_sale_condos

FULL ADDRESS,count
"2 EAST 55 STREET, 835",8
"2 EAST 55TH STREET, 935",7
"2 EAST 55 STREET, 921",7
"2 EAST 55 STREET, 936",6
"2 EAST 55 STREET, 903",6
"1420 YORK AVENUE, 5M",6
"140 7 AVENUE, 6A",6
"900 PARK AVENUE, 27E",5
"90 WILLIAM STREET, 2A",5
"845 UNITED NATIONS PLAZA, 24F",5


There are a lot less records with apt numbers, but still roughly 16K records enough to proceed.

In [ ]:
# Define a new table for condos with multiple sales. 

multi_sale_condos = multi_sale_condos.join(COL_FULL_ADDRESS, condos)
multi_sale_condos.sort(0)

FULL ADDRESS,count,SALE DATE,SALE PRICE,ADDRESS,APARTMENT NUMBER,YEAR BUILT,NEIGHBORHOOD,ZIP CODE,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY
"1 BOND STREET, 2B",2,2004-08-04 00:00:00,31000,1 BOND STREET,2B,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2B",2,2009-08-20 00:00:00,1591000,1 BOND STREET,2B,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2C",2,2017-08-18 00:00:00,5640000,1 BOND STREET,2C,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2C",2,2010-12-15 00:00:00,600000,1 BOND STREET,2C,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2D",2,2009-04-24 00:00:00,1195000,1 BOND STREET,2D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 2D",2,2014-01-07 00:00:00,1925000,1 BOND STREET,2D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 3A",2,2013-06-14 00:00:00,7200000,1 BOND STREET,3A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 3A",2,2008-06-06 00:00:00,5030000,1 BOND STREET,3A,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 4D",2,2006-08-30 00:00:00,1565000,1 BOND STREET,4D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS
"1 BOND STREET, 4D",2,2013-08-20 00:00:00,2415000,1 BOND STREET,4D,0,GREENWICH VILLAGE-CENTRAL,10012,R4,13 CONDOS - ELEVATOR APARTMENTS


## Find the first and last sale date for each condo

In [ ]:
purchase_dates = multi_sale_condos.select(COL_FULL_ADDRESS, COL_SALE_DATE).group([0], min)
sold_dates = multi_sale_condos.select(COL_FULL_ADDRESS, COL_SALE_DATE).group([0], max)

# Note for the purposes of this analysis, we can ignore any additnal sales between min and max dates

# Spot check data
purchase_dates.show(5)
sold_dates.show(5)

In [ ]:
# Update Labels

purchase_dates = purchase_dates.relabel(1, COL_PURCHASE_DATE)
sold_dates = sold_dates.relabel(1, COL_SOLD_DATE)

In [ ]:
# Join with Condos to get the sale price
purchase_dates = purchase_dates.join(COL_FULL_ADDRESS, condos, COL_FULL_ADDRESS).where(COL_SALE_DATE, are.equal_to, COL_PURCHASE_DATE)
purchase_dates = purchase_dates.select( COL_FULL_ADDRESS, COL_PURCHASE_DATE, COL_SALE_PRICE)


sold_dates = sold_dates.join(COL_FULL_ADDRESS, condos, COL_FULL_ADDRESS).where(COL_SALE_DATE, are.equal_to, COL_SOLD_DATE)
sold_dates = sold_dates.select( COL_FULL_ADDRESS, COL_SOLD_DATE, COL_SALE_PRICE)


In [ ]:
purchase_dates.show(5)
sold_dates.show(5)

# Hmm earlier we had 16187 now we have more! Could we have multiple sale records for the same date??

In [ ]:
# Yep duplicate sales on the same day!

purchase_dates.groups([COL_FULL_ADDRESS, COL_PURCHASE_DATE]).sort(2, descending=True).where(2, are.above(1))

In [ ]:
# Spot check some duplicate addresses to understand what's going on
condos.where(COL_FULL_ADDRESS, are.equal_to('2 EAST 55 STREET, 921'))

In [ ]:
# It's not really clear what might be going on. Let's just take the corresponding min and max value to keep moving along

purchase_dates = purchase_dates.group([COL_FULL_ADDRESS, COL_PURCHASE_DATE], min)
sold_dates = sold_dates.group([COL_FULL_ADDRESS, COL_SOLD_DATE], max)


In [ ]:
# Relabel and join the first and last sale tables to create a new condo sales table

purchase_dates = purchase_dates.relabel(2, COL_PURCHASE_PRICE)
sold_dates = sold_dates.relabel(2, COL_SOLD_PRICE)


condo_sales = purchase_dates.join(COL_FULL_ADDRESS, sold_dates, COL_FULL_ADDRESS)
condo_sales

In [ ]:
# Calculate Price and date Diffs

price_diffs = condo_sales.column(COL_SOLD_PRICE) - condo_sales.column(COL_PURCHASE_PRICE)
date_diffs = condo_sales.column(COL_SOLD_DATE) - condo_sales.column(COL_PURCHASE_DATE)

date_diffs = [ d.days for d in date_diffs ]


condo_sales = condo_sales.with_column( COL_PRICE_CHANGE, price_diffs, COL_PERIOD, date_diffs)
condo_sales.set_format(COL_PRICE_CHANGE, NumberFormatter())

In [ ]:
# Understand price difference data
col_stats(condo_sales, COL_PRICE_CHANGE)


There look to be some signifiant outliers here. The 99 percentile is 5 but the max is 44 MM a significant. The min value is -183MM, a 183 million dollor loss! 

Let's remove outliers so they dont imapact pricing analysis, but we definitely need to come back and look into what happened here. 

In [ ]:
# strip out the Price chnage outliers . 
percent_1 = percentile(1, price_diffs)
percent_99 = percentile(99, price_diffs)

largest_losses = condo_sales.where(COL_PRICE_CHANGE, are.below_or_equal_to(percent_1))
largest_gains = condo_sales.where(COL_PRICE_CHANGE, are.above_or_equal_to(percent_99))

condo_sales = condo_sales.where(COL_PRICE_CHANGE, are.between(percent_1, percent_99))
col_stats(condo_sales, COL_PRICE_CHANGE)

The data is starting to look like a normal distribution

In [ ]:
# Some sales are on recorded twice for the same date!

condo_sales.where( COL_PURCHASE_DATE, are.equal_to, COL_SOLD_DATE )


In [ ]:
# Let's spot check these

all_sales_data.where(COL_FULL_ADDRESS, are.equal_to('100 CENTRAL PARK SOUTH, 4B')).sort(0)

# These look to be duplicate records, Let's ignore for now

In [ ]:
# Ignore multiple sales on same date

condo_sales = condo_sales.where( COL_PURCHASE_DATE, are.not_equal_to, COL_SOLD_DATE )



## Issues with time between sales

Looking into the time between 2 sales for the same apartment we find some unusal data

In [ ]:
condo_sales.sort(COL_PERIOD)

Still Looks like there is some strange data where the sale dates are a few days apart, for most regular sale cycles we expect a gap of atleast 60-90 days. 
Let's ignore anything less that 3 months apart i.e. 90 days. 


In [ ]:
condo_sales = condo_sales.where( COL_PERIOD, are.above(90) )

## Time between sales

In [ ]:
col_stats(condo_sales, COL_PERIOD)

## Average Daily Price Change

Let's calulate the average daily price change to spot any other odd data. 

In [ ]:
daily_change = condo_sales.column(COL_PRICE_CHANGE) / condo_sales.column(COL_PERIOD) 

condo_sales = condo_sales.with_column(COL_DAILY_PRICE_CHANGE , daily_change ).sort(COL_DAILY_PRICE_CHANGE, descending=True)
condo_sales

In [ ]:
col_stats(condo_sales, COL_DAILY_PRICE_CHANGE)
draw_hist(condo_sales, COL_DAILY_PRICE_CHANGE, 1)

## There are some significant outliers

There could be a variety of reasons for the outliers. Perhaps they underpriced for when purchased and then corrected when sold later. Again I'd be intresting to investigate further, but for the purposes of this analysis let's ignore the outliers for now. 

In [ ]:
# strip out the Dailys Price change outliers. 
price_change_diffs = condo_sales.column(COL_DAILY_PRICE_CHANGE)

percent_1 = percentile(1, price_change_diffs)
percent_99 = percentile(99, price_change_diffs)

condo_sales = condo_sales.where(COL_DAILY_PRICE_CHANGE, are.between(percent_1, percent_99))


# Step 3: NYC Sale data analysis

# Understanding the data

Let's try to get an overview of the data by looking at the movement of the average sale price per year.

In [ ]:
years = [ d.year for d in condos.column(COL_SALE_DATE) ]

months = [ d.month for d in condos.column(COL_SALE_DATE) ]

condos = condos.with_column('SALE_YEAR', years, 'SALE_MONTH', months)

condo_mean = condos.select(COL_SALE_YEAR, COL_SALE_PRICE).group(COL_SALE_YEAR, np.mean).sort(0)
condo_mean.plot(COL_SALE_YEAR)



## Sales by neighbourhood 

Also helpful to usederstand how prices have changed in different neighborhoods. 

In [ ]:
neighborhoods = condos.group('NEIGHBORHOOD').sort(0).column(0)


def plot_neighborhood(neighborhood:str):
    '''Plot the average sale for a specified neign'''
    condos.where('NEIGHBORHOOD', are.equal_to(neighborhood)).select(COL_SALE_YEAR, COL_SALE_PRICE).group(0, np.mean).plot(0, label=neighborhood)
    plots.title = neighborhood
    plots.plot(condo_mean.column(0), condo_mean.column(1), color='gold', label='' )
    return

ignnore = interact(plot_neighborhood, neighborhood=neighborhoods)


## Sampling

Let's take a deeper dive at apartment sales in 2010 as a sample. Note: we could have selected any range. This is just a random selection to reduce the noise in the data



In [ ]:
sales_2010 = condo_sales.where(COL_PURCHASE_DATE, are.between( datetime.datetime(2010, 1, 1), datetime.datetime(2010, 12, 31)))

Table().with_columns(
    'PERIOD',  sales_2010.column(COL_PERIOD), 
    'PRICE CHANGE', sales_2010.column(COL_PRICE_CHANGE)
).scatter(0, 1, fit_line=True)

print('Correlation betweeen Price Change and Time: ', correlation(sales_2010, COL_PERIOD, COL_PRICE_CHANGE))

That's a low correlation, I was expectating a closer relationship that is roghly keeping tracking with avg overall sale price we plotted earlier.


Ok, let's look at the correlation between the first and last sale price. 

In [ ]:
Table().with_columns(
    'PURCHASED PRICE',  sales_2010.column(COL_PURCHASE_PRICE), 
    'SOLD PRICE', sales_2010.column(COL_SOLD_PRICE)
).scatter(0, fit_line=True)

correlation(sales_2010, COL_PURCHASE_PRICE, COL_SOLD_PRICE)



Wow that's a really high correlation. According to this we could predict the last sale price of a property, just based on it's first sale price. i.e. independent of the time between sales! 

Somehting doesn't seem right, we know that the market moves over time. Let's dig in a little deeper and plot the residuals

In [ ]:
a = slope(condo_sales, COL_PURCHASE_PRICE, COL_SOLD_PRICE)
b = intercept(condo_sales, COL_PURCHASE_PRICE, COL_SOLD_PRICE)

first_prices = condo_sales.column(COL_PURCHASE_PRICE)

predicted = first_prices * a + b

errors = condo_sales.column(COL_SOLD_PRICE) - predicted


Table().with_columns(
    'PURCHASE PRICE',  condo_sales.column(COL_PURCHASE_PRICE), 
    'ERRORS', errors
).scatter(0, fit_line=True)



The residuals are not evenly spread put, also some large outliers are skewing the results so a liner regression model isn't the right approach here.

Let's take a deeper look into a particular price bands to see what could be going on. 

In [ ]:
price_band = condo_sales.where(COL_PURCHASE_PRICE, are.between(750000, 1000000))

price_band.scatter(COL_PURCHASE_PRICE, COL_SOLD_PRICE, fit_line=True)
correlation(price_band, COL_PURCHASE_PRICE, COL_SOLD_PRICE)

Now the correlation is much lower, The data is more spread out. Also a average seems to skew a little higher do the outliers a visible in the chart.

## Removing Outliers

Let's look at the purchase price outliers

In [ ]:
col_stats(condo_sales, COL_PURCHASE_PRICE)

Let's remove the ouliers so we can get a better picture of the data. 

In [ ]:
purchase_prices = condo_sales.column(COL_PURCHASE_PRICE)

percent_1 = percentile(1, purchase_prices)
percent_99 = percentile(99, purchase_prices)

condo_sales = condo_sales.where(COL_PURCHASE_PRICE, are.between(percent_1, percent_99))
col_stats(condo_sales, COL_PURCHASE_PRICE)

much better!

# Price change % 

Now let's calculate the price change as a % of the purchase price.

In [ ]:
percents = sales_2010.column(COL_PRICE_CHANGE) / sales_2010.column(COL_PURCHASE_PRICE) * 100

COL_PRICE_PERCENT = 'PRICE CHANGE %'

sales_2010 = sales_2010.with_column(COL_PRICE_PERCENT, percents)



## Sanitize price change % 

Looking into how  the price change % data is distributed. Again we need to clean out the outliers.

In [ ]:

col_stats(sales_2010, COL_PRICE_PERCENT)
draw_hist(sales_2010, COL_PRICE_PERCENT, 2)

In [ ]:
# strip out the Price Percent change outliers. 

price_changes = sales_2010.column(COL_PRICE_PERCENT)

percent_2 = percentile(2, price_changes)
percent_98 = percentile(98, price_changes)

sales_2010 = sales_2010.where(COL_PRICE_PERCENT, are.between(percent_2, percent_98))



In [ ]:
sales_2010.scatter(COL_PERIOD, COL_PRICE_PERCENT, fit_line=True)
correlation(sales_2010, COL_PERIOD, COL_PRICE_PERCENT)

Ok there there looks to be an upward trend here and prices changes increase over time. That said the correlation isn't linear. Smoothing the data into monthly (30) intervals look like the following

In [ ]:

periods = sales_2010.column(COL_PERIOD)

min_period = min(periods)
max_period = max(periods)


period_groups = []
period_sales = []

for i in np.arange(min_period, max_period, 30 ):
    period_groups.append(i)
    period_sales.append(np.mean(sales_2010.where(COL_PERIOD, are.between(i, i+30)).column(COL_PRICE_PERCENT)))
    

Table().with_columns(
    COL_PERIOD,  period_groups, 
    COL_PRICE_PERCENT, period_sales
).scatter(0)

Defintitely shows a trend! 

Let's take a look at other years

In [ ]:
# Trim the outliers. 
percents = condo_sales.column(COL_PRICE_CHANGE) / condo_sales.column(COL_PURCHASE_PRICE) * 100

condo_sales = condo_sales.with_column(COL_PRICE_PERCENT, percents)

percent_1 = percentile(1, percents)
percent_99 = percentile(99, percents)

condo_sales = condo_sales.where(COL_PRICE_PERCENT, are.between(percent_1, percent_99))

def plot_price_change_year(year):
    ''' Plot the price change % for a given year'''
    valid_sales = condo_sales.where(COL_PURCHASE_DATE, are.between_or_equal_to( datetime.datetime(year, 1, 1), datetime.datetime(year+1, 1, 1)))

    min_period = min(periods)
    max_period = max(periods)


    period_groups = []
    period_sales = []

    for i in np.arange(min_period, max_period, 30 ):
        period_groups.append(i)
        period_sales.append(np.mean(valid_sales.where(COL_PERIOD, are.between(i, i+30)).column(COL_PRICE_PERCENT)))

    Table().with_columns(
        COL_PERIOD,  period_groups, 
        COL_PRICE_PERCENT, period_sales
    ).scatter(0)

interact(plot_price_change_year, year=np.arange(2003,2019) )



We can now say there is a a correlation between the price change and time reflective of the overall movement in the market over time. 

Let's create some prediction models. 

# Step 4: Predicting NYC property prices

## Prediction model: nearest neighbor

Our prediction model should use both the purchase price and date for prediction. Let's create it now.

We will first convert purchase date, price and sold dates into standard units so they can be used to compute distance. 

There we will create the training and testing set.

In [ ]:
# Convert to standard units
purchase_dates_timestamps = [ date.timestamp() for date in condo_sales.column(COL_PURCHASE_DATE)]
purchase_dates_su = standard_units(purchase_dates_timestamps)

sold_dates_timestamps = [ date.timestamp() for date in condo_sales.column(COL_SOLD_DATE)]
sold_dates_su = standard_units(sold_dates_timestamps)

purchase_price_su = standard_units(condo_sales.column(COL_PURCHASE_PRICE))

condo_sales_su = condo_sales.with_column(
    COL_PURCHASE_DATE_SU, purchase_dates_su, 
    COL_PURCHASE_PRICE_SU, purchase_price_su, 
    COL_SOLD_DATE_SU, sold_dates_su,
)

# Create the training and testing sets.
training_sales, test_sales = condo_sales_su.split(int(condo_sales.num_rows * 0.6))
training_sales


In [ ]:
condo_sales_su.to_df().to_csv( output_directory + 'condo_sales.csv', index = False)


In [ ]:
# Columns used to the calculate the distance between two point i.e. 2 properties that were purchased and sold. 
# We picked the purchase date & price and the sold date converted to standard units as the important columns
# to use for calculating the distance.
distance_columns = [COL_PURCHASE_DATE_SU, COL_PURCHASE_PRICE_SU, COL_SOLD_DATE_SU]



def all_distances(training, new_point):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.select(distance_columns)
    return distance.cdist( attributes.to_array().tolist(), [new_point]).flatten()

def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

def closest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def estimate(training, purchase_point, k):
    """Estimate a price based on nearest neighbours"""
    close_points = closest(training_sales, purchase_point, k)
    avg_price_change = np.mean(close_points.column(COL_PRICE_PERCENT))
    return avg_price_change

def evaluate_accuracy(training, test, k):
    """Evalute the accuracy of the model generating using training data on test data"""
    # select the columns to compare
    test_attributes = test.select(distance_columns)

    # compute the predicted change for each test row
    def price_testrow(row):
        return estimate(training, row, k)

    # Calculate the predicted price and error
    c = test_attributes.apply(price_testrow)
    
    estimated = test[COL_PURCHASE_PRICE] * (100 + c )/100
    error = (test[COL_SOLD_PRICE] - estimated  ) / test[COL_SOLD_PRICE] * 100
    
    return test.with_column("Estimated", estimated, 'Error %', error)

In [ ]:

sample_sales =  test_sales
estimates = evaluate_accuracy(training_sales, sample_sales, 10)


In [ ]:
estimates.scatter(COL_SOLD_PRICE, 'Error %')

col_stats(estimates, "Error %")

In [ ]:
def predict_sale_price(purchase_price, purchase_date: datetime.date):
    """Predict the current value based on the purchase price and purchase date."""
    
    purchase_date_timestamp = time.mktime(purchase_date.timetuple())

    # Set sold date to now
    sold_date = datetime.datetime.now().timestamp()

    # calcuate all values in standard units.
    purchase_date_su = (purchase_date_timestamp - np.mean(purchase_dates_timestamps)) / np.std(purchase_dates_timestamps)
    purchase_price_su = (purchase_price - np.mean(condo_sales.column(COL_PURCHASE_PRICE))) / np.std(condo_sales.column(COL_PURCHASE_PRICE))
    sold_date_su = (sold_date - np.mean(sold_dates_timestamps)) / np.std(sold_dates_timestamps)

    # debugging
    # print(purchase_date_su, purchase_price_su, sold_date_su)

    # create a target row to get estimates
    target_row = Table(distance_columns)
    target_row = target_row.with_row([purchase_date_su,purchase_price_su,sold_date_su ])
    
    price_change_percent = estimate(training_sales, target_row.row(0), 10)
    
    price = purchase_price * (1 + price_change_percent / 100)
    
    display(HTML("<H1>Current Price: ${:,.0f} </H1>".format(price)))
#     display(HTML("<H2>Estimated Price Change: {:.2f}% </H2>".format(price_change_percent)))
    
    display(HTML("<H1>Compareable sales</H1>"))
    closest(training_sales, target_row.row(0), 10).drop(distance_columns).show()
    
style = {'description_width': 'initial'}

purchase_price_slider = widgets.IntSlider(
    description="Purchase Price",
    value=750000,
    min=100000,
    max=6000000,
    step=10000,
    disabled=False,
    continuous_update=False,
    style=style
    
)

purchase_date_picker = widgets.DatePicker(
    description='Purchase Date',
    disabled=False,
    style=style,
    value=datetime.date(2010, 1, 1)
)

# Price Predictor

Enter the **purchase price and date** of an apartment to see it's current price and comparable sales

In [ ]:
style = {'description_width': 'initial'}
IntSlider(description='A too long description', style=style)

_ = interact( predict_sale_price, purchase_price=purchase_price_slider, purchase_date=purchase_date_picker, )

# Furhter Explorations

It would be interesting furhter explore this data by:

1. Applying different weights to the distanace calculations. e.g. The purchase price should be more important than the other attributes. 
2. Including location when calculating the distance between two properties. 
3. Including co-ops.

If you have suggestions or thoughts, I would appreciate if you share them using the questions section below. 

Thank you for reading!